### Import librerie

In [1]:
import nltk
#nltk.download('punkt')
from nltk import word_tokenize 
from nltk.corpus import stopwords
#nltk.download('stopwords')
#nltk.download('words')
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
from time import time

In [3]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np

In [4]:
import gensim
from gensim.models import LdaSeqModel

### Caricamento dataset

In [51]:
df = pd.read_csv('C:/Users/micky/OneDrive/Desktop/Tesi DS/data/NIPS.csv')
df.head()

,id,year,title,event_type,pdf_name,abstract,paper_text
0,1,1987,Self-Organization of Associative Database and ...,NaN,1-self-organization-of-associative-database-an...,Abstract Missing,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...
1,10,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,10-a-mean-field-theory-of-layer-iv-of-visual-c...,Abstract Missing,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...
2,100,1988,Storing Covariance by the Associative Long-Ter...,NaN,100-storing-covariance-by-the-associative-long...,Abstract Missing,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...
3,1000,1994,Bayesian Query Construction for Neural Network...,NaN,1000-bayesian-query-construction-for-neural-ne...,Abstract Missing,Bayesian Query Construction for Neural\nNetwor...
4,1001,1994,"Neural Network Ensembles, Cross Validation, an...",NaN,1001-neural-network-ensembles-cross-validation...,Abstract Missing,"Neural Network Ensembles, Cross\nValidation, a..."


In [52]:
df = df[df['abstract']!= 'Abstract Missing']

In [53]:
df.shape

(3924, 7)

### Pre-processing

In [54]:
df = df[['year', 'abstract']]

In [55]:
import random
random.seed(1234)
df = df.sample(frac=0.02, replace=False, random_state=1) #campionamento del dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78 entries, 3738 to 2435
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   year      78 non-null     int64 
 1   abstract  78 non-null     object
dtypes: int64(1), object(1)
memory usage: 1.8+ KB


In [56]:
df['abstract'] = df['abstract'].str.lower() #minuscolo 
df['abstract'] = df['abstract'].str.replace(r'[0-9]+', ' ') #rimozione numeri
df['abstract'] = df['abstract'].str.replace(r'http\S+', ' ') #rimozione link
df['abstract'] = df['abstract'].str.replace(r'[^a-zA-Z0-9 ]', ' ') #rimozione caratteri speciali
df['abstract'] = df['abstract'].str.replace('\\b\\w{1,2}\\b', ' ') #rimozione singole lettere
df['abstract'] = df['abstract'].str.replace(' +', ' ') #rimozione spazi extra

In [58]:
df = df.sort_values(by=['year'])  #ordinamento cronologico necessario per il DTM
df.head()

,year,abstract
2435,2007,new algorithm for line learning linear thresh...
2415,2007,propose novel method for linear dimensionalit...
2894,2008,recent work long and servedio short presented...
2891,2008,extensive labeled data for image annotation sy...
2677,2008,for many supervised learning problems possess ...


In [59]:
df = df[df['abstract'].map(lambda x: len(x)!=0)]

In [60]:
series_slices = df.groupby(["year"]).size().values
series_slices

array([ 2,  6,  6,  4,  8, 10,  3,  6,  8, 12, 13], dtype=int64)

In [61]:
corp = df['abstract']

In [204]:
stemmer = SnowballStemmer(language='english')

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems


vect = CountVectorizer(tokenizer=tokenize, 
                       stop_words='english', 
                       lowercase=True) 


X = vect.fit_transform(corp.values.astype('U')) #astype per convertire in stringa

doc_term_matrix = pd.DataFrame(X.toarray(), columns=vect.get_feature_names())

doc_term_matrix.head()

,abil,abl,absolut,abstract,accept,access,accomplish,accord,account,accur,...,worst,wpdl,writer,written,wrong,year,yeast,yield,zero,zeta
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [205]:
corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False) 
id_map = dict((v, k) for k, v in vect.vocabulary_.items())

In [206]:
t0 = time()
ldaseq = LdaSeqModel(corpus=corpus, time_slice=series_slices, id2word=id_map, num_topics=3)
print("done in %0.3fs." % (time() - t0))

done in 249.088s.


In [207]:
first = ldaseq.print_topic_times(0)

In [208]:
a = pd.DataFrame(zip(*first), columns=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J' ,'K'])

In [209]:
a.head()

,A,B,C,D,E,F,G,H,I,J,K
0,"(model, 0.019783964709751533)","(model, 0.01991362147077083)","(model, 0.019782540129761602)","(model, 0.019384624501216825)","(model, 0.019080347822184835)","(model, 0.018919145648533475)","(model, 0.019076345852284418)","(model, 0.01929624651794585)","(model, 0.01925674324537372)","(model, 0.018735886114970728)","(model, 0.0183839067841869)"
1,"(algorithm, 0.017029448901261568)","(algorithm, 0.01671903324074084)","(algorithm, 0.01664276276171765)","(algorithm, 0.016528179253396693)","(algorithm, 0.016412663854600777)","(algorithm, 0.016437872445148386)","(algorithm, 0.016616385095933633)","(algorithm, 0.016814128321436857)","(algorithm, 0.017074326192175707)","(algorithm, 0.017412157712469126)","(algorithm, 0.017736131770398213)"
2,"(optim, 0.015043820392849797)","(optim, 0.01512066857474488)","(optim, 0.015317843716232883)","(optim, 0.015290432307833043)","(optim, 0.01503217973806444)","(optim, 0.014841161563516135)","(optim, 0.014757011886767578)","(optim, 0.0146215487152987)","(optim, 0.014551559252003108)","(optim, 0.014300783638271725)","(optim, 0.014141282687264296)"
3,"(data, 0.012162327535835064)","(data, 0.012167701327750776)","(use, 0.012138297998110276)","(data, 0.012104977398921319)","(data, 0.012105090007046633)","(data, 0.012202671822848024)","(data, 0.012455634224546585)","(data, 0.01265515431215726)","(data, 0.012785392946373363)","(data, 0.01279605001286679)","(data, 0.012800889637853974)"
4,"(use, 0.012055003679220725)","(use, 0.012109762237874397)","(data, 0.01208627732030907)","(use, 0.011898929503357504)","(use, 0.01164333733860615)","(use, 0.01143884665168838)","(use, 0.011259130401910697)","(use, 0.011030934406155246)","(use, 0.010834797121635398)","(use, 0.010729961507380592)","(use, 0.010610095622559606)"


In [210]:
a['A.x'], a['A.y'] = zip(*a.A)
a['B.y'] = list(zip(*a.B))[1]
a['C.y'] = list(zip(*a.C))[1]
a['D.y'] = list(zip(*a.D))[1]
a['E.y'] = list(zip(*a.E))[1]
a['F.y'] = list(zip(*a.F))[1]
a['G.y'] = list(zip(*a.G))[1]
a['H.y'] = list(zip(*a.H))[1]
a['I.y'] = list(zip(*a.I))[1]
a['J.y'] = list(zip(*a.J))[1]
a['K.y'] = list(zip(*a.K))[1]

In [211]:
a = a.drop(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K'], axis=1, errors='ignore')

In [212]:
new = pd.DataFrame(a.iloc[:, 0:12].values, columns=['terms', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017'])
new.head()

,terms,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,model,0.019784,0.019914,0.019783,0.019385,0.01908,0.018919,0.019076,0.019296,0.019257,0.018736,0.018384
1,algorithm,0.017029,0.016719,0.016643,0.016528,0.016413,0.016438,0.016616,0.016814,0.017074,0.017412,0.017736
2,optim,0.015044,0.015121,0.015318,0.01529,0.015032,0.014841,0.014757,0.014622,0.014552,0.014301,0.014141
3,data,0.012162,0.012168,0.012138,0.012105,0.012105,0.012203,0.012456,0.012655,0.012785,0.012796,0.012801
4,use,0.012055,0.01211,0.012086,0.011899,0.011643,0.011439,0.011259,0.011031,0.010835,0.01073,0.01061


In [213]:
#for topic 0
topic0_evolution = pd.DataFrame()
for i in range(2007, 2018, 1):
    topic0_evolution['time_{}'.format(i)] = list(zip(*ldaseq.print_topic_times(topic=0,top_terms=30)[i-2007]))[0]

In [214]:
topic0_evolution

,time_2007,time_2008,time_2009,time_2010,time_2011,time_2012,time_2013,time_2014,time_2015,time_2016,time_2017
0,model,model,model,model,model,model,model,model,model,model,model
1,algorithm,algorithm,algorithm,algorithm,algorithm,algorithm,algorithm,algorithm,algorithm,algorithm,algorithm
2,optim,optim,optim,optim,optim,optim,optim,optim,optim,optim,optim
3,data,data,use,data,data,data,data,data,data,data,data
4,use,use,data,use,use,use,use,use,use,use,use
5,predict,predict,predict,predict,predict,predict,predict,predict,learn,learn,learn
6,learn,learn,learn,learn,learn,learn,learn,learn,predict,predict,predict
7,dynam,dynam,dynam,dynam,dynam,dynam,dynam,dynam,dynam,dynam,dynam
8,base,base,base,base,base,base,base,effici,effici,effici,effici
9,effici,effici,effici,effici,effici,effici,effici,base,base,base,base


In [215]:
#for topic 1
topic1_evolution = pd.DataFrame()
for i in range(2007, 2018, 1):
    topic1_evolution['time_{}'.format(i)] = list(zip(*ldaseq.print_topic_times(topic=1,top_terms=30)[i-2007]))[0]

In [216]:
topic1_evolution

,time_2007,time_2008,time_2009,time_2010,time_2011,time_2012,time_2013,time_2014,time_2015,time_2016,time_2017
0,model,model,model,model,model,model,model,model,model,model,model
1,learn,learn,learn,learn,learn,learn,learn,learn,learn,learn,learn
2,approach,approach,approach,approach,approach,approach,approach,approach,use,use,use
3,use,use,use,use,use,use,use,use,approach,approach,approach
4,data,data,data,data,data,data,data,data,data,data,data
5,method,method,method,method,method,method,method,method,label,method,method
6,label,label,label,label,label,label,label,label,method,label,label
7,algorithm,function,function,function,function,function,function,function,function,function,embed
8,function,algorithm,distribut,distribut,distribut,distribut,distribut,distribut,estim,embed,function
9,distribut,distribut,algorithm,algorithm,algorithm,estim,estim,estim,distribut,estim,estim


In [217]:
#for topic 2
topic2_evolution = pd.DataFrame()
for i in range(2007, 2018, 1):
    topic2_evolution['time_{}'.format(i)] = list(zip(*ldaseq.print_topic_times(topic=2,top_terms=30)[i-2007]))[0]

In [218]:
topic2_evolution

,time_2007,time_2008,time_2009,time_2010,time_2011,time_2012,time_2013,time_2014,time_2015,time_2016,time_2017
0,algorithm,algorithm,algorithm,algorithm,algorithm,algorithm,algorithm,algorithm,algorithm,algorithm,algorithm
1,learn,learn,learn,learn,learn,learn,problem,problem,problem,problem,problem
2,problem,problem,problem,problem,problem,problem,learn,learn,matrix,matrix,matrix
3,result,result,result,result,result,matrix,matrix,matrix,learn,learn,learn
4,matrix,matrix,matrix,matrix,matrix,result,result,result,result,result,estim
5,estim,model,model,model,model,estim,estim,estim,estim,estim,result
6,data,data,estim,estim,estim,model,model,model,model,model,mat
7,model,estim,data,data,data,data,data,data,mat,mat,model
8,high,high,propos,propos,propos,propos,propos,function,rank,function,rank
9,propos,propos,high,high,high,high,high,high,data,rank,high


In [219]:
lista0 = []
for i in topic0_evolution.columns:
    if((topic0_evolution[i]=='time').any()):
        lista0 += [topic0_evolution.index[topic0_evolution[i] == 'time'].tolist()[0]]

In [220]:
lista0

[27, 27, 27, 27, 27, 26, 25, 25, 25, 25, 21]

In [223]:
lista1 = []
for i in topic1_evolution.columns:
    if((topic1_evolution[i]=='network').any()):
        lista1 += [topic1_evolution.index[topic1_evolution[i] == 'network'].tolist()[0]]

In [224]:
lista1

[27, 27, 26, 24, 24, 24, 23, 22, 21, 20, 17]

In [229]:
lista2 = []
for i in topic2_evolution.columns:
    if((topic2_evolution[i]=='cluster').any()):
        lista2 += [topic2_evolution.index[topic2_evolution[i] == 'cluster'].tolist()[0]]

In [230]:
lista2

[26, 25, 26, 26, 26, 27, 26, 26, 26, 23, 24]